## Car Color Detection


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical # pyright: ignore[reportMissingImports]
from sklearn.model_selection import train_test_split


In [ ]:
import kagglehub # pyright: ignore[reportMissingImports]


path = kagglehub.dataset_download("landrykezebou/vcor-vehicle-color-recognition-dataset")

print("Path to dataset files:", path)

100%|██████████| 574M/574M [00:28<00:00, 21.2MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1


In [ ]:
dataset_path = "/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1"



In [ ]:
import os
import pandas as pd # pyright: ignore[reportMissingModuleSource]

dataset_path = "/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1"
folders = ["train", "val", "test"]

data = []

for folder in folders:
    folder_path = os.path.join(dataset_path, folder)
    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_folder):
            continue
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            # Define labels
            blue = 1 if class_name == "blue" else 0
            other = 1 if class_name != "blue" else 0
            data.append([img_path, blue, other])

# Create DataFrame
df = pd.DataFrame(data, columns=["image_path", "blue", "other"])

# Show first few rows
print(df.head())
print("Total samples:", len(df))
print("Blue images:", df['blue'].sum())
print("Other images:", df['other'].sum())


                                          image_path  blue  other
0  /root/.cache/kagglehub/datasets/landrykezebou/...     1      0
1  /root/.cache/kagglehub/datasets/landrykezebou/...     1      0
2  /root/.cache/kagglehub/datasets/landrykezebou/...     1      0
3  /root/.cache/kagglehub/datasets/landrykezebou/...     1      0
4  /root/.cache/kagglehub/datasets/landrykezebou/...     1      0
Total samples: 10373
Blue images: 1060
Other images: 9313


In [ ]:
# cnn_blue_other.py
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# ---------------- Parameters ----------------
IMG_SIZE = 64
BATCH_SIZE = 32
EPOCHS = 15
dataset_path = "/root/.cache/kagglehub/datasets/landrykezebou/vcor-vehicle-color-recognition-dataset/versions/1"
folders = ["train", "val", "test"]

# ---------------- Create DataFrame ----------------
data = []
for folder in folders:
    folder_path = os.path.join(dataset_path, folder)
    for class_name in os.listdir(folder_path):
        class_folder = os.path.join(folder_path, class_name)
        if not os.path.isdir(class_folder):
            continue
        for img_name in os.listdir(class_folder):
            img_path = os.path.join(class_folder, img_name)
            blue = 1 if class_name == "blue" else 0
            other = 1 if class_name != "blue" else 0
            label = 0 if class_name == "blue" else 1  # 0=blue, 1=other
            data.append([img_path, label])

df = pd.DataFrame(data, columns=["image_path", "label"])
print("Total samples:", len(df))
print("Blue images:", sum(df['label']==0))
print("Other images:", sum(df['label']==1))

# ---------------- Load Images ----------------
images = []
labels = []

for idx, row in df.iterrows():
    img = cv2.imread(row['image_path'])
    if img is not None:
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img)
        labels.append(row['label'])

X = np.array(images, dtype="float32") / 255.0
y = to_categorical(np.array(labels), num_classes=2)

# ---------------- Train/Test Split ----------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Training samples:", len(X_train), "Testing samples:", len(X_test))

# ---------------- CNN Model ----------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: blue, other
])

model.compile(optimizer=Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# ---------------- Train Model ----------------
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE)

# ---------------- Save Model ----------------
model.save("car_color_blue_other.h5")
print("Model saved as car_color_blue_other.h5")


Total samples: 10373
Blue images: 1060
Other images: 9313
Training samples: 8298 Testing samples: 2075


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - accuracy: 0.9345 - loss: 0.1599 - val_accuracy: 0.9802 - val_loss: 0.0577
Epoch 2/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9797 - loss: 0.0645 - val_accuracy: 0.9846 - val_loss: 0.0460
Epoch 3/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9874 - loss: 0.0422 - val_accuracy: 0.9875 - val_loss: 0.0326
Epoch 4/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9853 - loss: 0.0456 - val_accuracy: 0.9870 - val_loss: 0.0339
Epoch 5/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9872 - loss: 0.0363 - val_accuracy: 0.9884 - val_loss: 0.0324
Epoch 6/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9839 - loss: 0.0437 - val_accuracy: 0.9908 - val_loss: 0.0285
Epoch 7/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9901 - loss: 0.0322 - val_accuracy: 0.9908 - val_loss: 0.0274
Epoch 8/15
260/260 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9903 - loss: 0.0306 - val_accuracy: 

Model saved as car_color_blue_other.h5
